In [71]:
import pathlib
from gemd.entity import ProcessTemplate, ParameterTemplate, RealBounds, CategoricalBounds, ConditionTemplate
from gemd.entity import ProcessSpec, PerformedSource, Condition, NominalReal, NominalCategorical, Parameter
from gemd.entity.util import make_instance
from gemd.json import GEMDJson

In [98]:
#One cell to define a dictionary of all the parameters

creating_puck = {
    'name': 'Julia',
    'date': '7/15/2022',
    'glass-slide thickness': '2.0', # in units of mm
    'vaseline name': 'Vaspure',
    'amount of vaseline': '6', # in units of l
    'humidity': '0.08', # fractional, from 0 to 1
}

In [99]:
humidity_templ = ConditionTemplate(
    'humidity',
    bounds=RealBounds(0.,1.0,'dimensionless')
)

temp = ProcessTemplate(
    'creating puck',
    parameters=[
        ParameterTemplate(
            'glass thickness',
            bounds=RealBounds(0.0,10.0,'mm')
        ),
        ParameterTemplate(
            'amount of vaseline',
            bounds=RealBounds(0.0,15.0,'ml')
        ),
        ParameterTemplate(
            'name of vaseline',
            bounds=CategoricalBounds(['Vaspure','Vaseline','Giant','Nivea'])
        )
    ],
)

In [100]:
temp

{'parameters': [[{'description': None, 'name': 'glass thickness', 'tags': [], 'uids': {}, 'bounds': {'default_units': 'millimeter', 'lower_bound': 0.0, 'upper_bound': 10.0, 'type': 'real_bounds'}, 'type': 'parameter_template'}, None], [{'description': None, 'name': 'amount of vaseline', 'tags': [], 'uids': {}, 'bounds': {'default_units': 'milliliter', 'lower_bound': 0.0, 'upper_bound': 15.0, 'type': 'real_bounds'}, 'type': 'parameter_template'}, None], [{'description': None, 'name': 'name of vaseline', 'tags': [], 'uids': {}, 'bounds': {'type': 'categorical_bounds', 'categories': ['Giant', 'Nivea', 'Vaseline', 'Vaspure']}, 'type': 'parameter_template'}, None]], 'allowed_names': [], 'allowed_labels': [], 'description': None, 'name': 'creating puck', 'tags': [], 'conditions': [], 'uids': {}, 'type': 'process_template'}

In [101]:
spec = ProcessSpec(
    name='creating puck',
    template=temp,
    conditions=[],
    parameters=[
        Parameter(
            'glass thickness',
            value=NominalReal(float(creating_puck['glass-slide thickness']),'mm'),
        ),
        Parameter(
            'amount of vaseline',
            value=NominalReal(float(creating_puck['amount of vaseline']),'ml')
        ),
        Parameter(
            'name of vaseline',
            value=NominalCategorical(creating_puck['vaseline name'])
        )
    ],
)

In [102]:
spec

{'parameters': [{'value': {'units': 'millimeter', 'nominal': 2.0, 'type': 'nominal_real'}, 'template': None, 'name': 'glass thickness', 'origin': 'unknown', 'notes': None, 'file_links': [], 'type': 'parameter'}, {'value': {'units': 'milliliter', 'nominal': 6.0, 'type': 'nominal_real'}, 'template': None, 'name': 'amount of vaseline', 'origin': 'unknown', 'notes': None, 'file_links': [], 'type': 'parameter'}, {'value': {'category': 'Vaspure', 'type': 'nominal_categorical'}, 'template': None, 'name': 'name of vaseline', 'origin': 'unknown', 'notes': None, 'file_links': [], 'type': 'parameter'}], 'template': {'parameters': [[{'description': None, 'name': 'glass thickness', 'tags': [], 'uids': {}, 'bounds': {'default_units': 'millimeter', 'lower_bound': 0.0, 'upper_bound': 10.0, 'type': 'real_bounds'}, 'type': 'parameter_template'}, None], [{'description': None, 'name': 'amount of vaseline', 'tags': [], 'uids': {}, 'bounds': {'default_units': 'milliliter', 'lower_bound': 0.0, 'upper_bound':

In [103]:
run = make_instance(spec)
run.source = PerformedSource(creating_puck['name'],creating_puck['date'])
run.conditions.append(
    Condition(
            'humidity',
            value=NominalReal(float(creating_puck['humidity']),'kg/kg'),
            template=humidity_templ
        )
)

In [104]:
run

{'spec': {'parameters': [{'value': {'units': 'millimeter', 'nominal': 2.0, 'type': 'nominal_real'}, 'template': None, 'name': 'glass thickness', 'origin': 'unknown', 'notes': None, 'file_links': [], 'type': 'parameter'}, {'value': {'units': 'milliliter', 'nominal': 6.0, 'type': 'nominal_real'}, 'template': None, 'name': 'amount of vaseline', 'origin': 'unknown', 'notes': None, 'file_links': [], 'type': 'parameter'}, {'value': {'category': 'Vaspure', 'type': 'nominal_categorical'}, 'template': None, 'name': 'name of vaseline', 'origin': 'unknown', 'notes': None, 'file_links': [], 'type': 'parameter'}], 'template': {'parameters': [[{'description': None, 'name': 'glass thickness', 'tags': [], 'uids': {}, 'bounds': {'default_units': 'millimeter', 'lower_bound': 0.0, 'upper_bound': 10.0, 'type': 'real_bounds'}, 'type': 'parameter_template'}, None], [{'description': None, 'name': 'amount of vaseline', 'tags': [], 'uids': {}, 'bounds': {'default_units': 'milliliter', 'lower_bound': 0.0, 'uppe

In [110]:
encoder = GEMDJson()

fn = 'example_process_template.json'
with open(pathlib.Path()/fn,'w') as fp :
    fp.write(encoder.thin_dumps(temp,indent=2))

fn = 'example_process_spec.json'
with open(pathlib.Path()/fn,'w') as fp :
    fp.write(encoder.thin_dumps(spec,indent=2))

fn = 'example_process_run.json'
with open(pathlib.Path()/fn,'w') as fp :
    fp.write(encoder.thin_dumps(run,indent=2))

entire_model_fn = 'entire_model.json'
with open(pathlib.Path()/entire_model_fn,'w') as fp :
    fp.write(encoder.dumps([run],indent=2))


In [115]:
with open(pathlib.Path()/entire_model_fn,'r') as fp :
    runs = encoder.load(fp)
run = runs[0]

In [117]:
run.spec==spec

True

## Original code

In [6]:
#name of of material being placed into a copper X-ray diffractometer
XRD_Cu_name = ' '
#conditions 
XRD_Cu_conditions = {
    'glass-slide thickness': '2.0mm',
    'vasline name': 'Vaspure',
    'amount of vasilne': '6ml',
    'humidity level in lab': '20ml'}
#parameters - put in name of parameter file
XRD_Cu_parameters = {
    'angle range': None,
    'duration': None}
#tags
XRD_Cu_tags = ''
#notes
XRD_Cu_notes = ''
#links to the file
XRD_Cu_filelinks = ''


In [4]:
#name of of material being placed into a Electric Transport PPMS diffractometer
PPMS_ET_name = ' '
#conditions 
PPMS_ET_conditions = {
    'voltage orientation': None,
    'humidity level in lab': None }
#parameters - put in name of parameter file
PPMS_ET_parametersfile = {
    'current': None,
    'sample dimentions': None,
    'compound stochiometry': None}
#tags
PPMS_ET_tags = ''
#notes
PPMS_ET_notes = ''
#links to the file
PPMS_ET_filelinks = ''

In [7]:
from gemd.entity.object import ProcessSpec


spec = ProcessSpec(XRD_Cu_name,XRD_Cu_conditions,XRD_Cu_parameters,XRD_Cu_tags,XRD_Cu_notes,XRD_Cu_filelinks)
# where each of the parameters to the function call are filled out with information from the dictionary. 

#The name is the name of the material (a string), conditions is a list of GEMD Condition objects, etc.
# and then when you have that you can say:

run = make_instance(spec)
run.source = PerformedSource(person,date)

TypeError: __init__() takes 2 positional arguments but 7 were given